In [8]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
caminho_arquivo = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/07-Santander/Input/2025-03-planilhaExtrato.xls'
output = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/07-Santander/Output/extrato_formatado.csv'
pasta_input = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/07-Santander/Input'

In [2]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Credenciais do banco de dados
dbname = "neondb"
user = "neondb_owner"
password = "npg_ayeAjRZ8sPD7"
host = "ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech"
port = "5432"

# Criar a string de conexão com o PostgreSQL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Criar a conexão com o banco de dados
engine = create_engine(connection_string)

In [9]:
import pandas as pd
import os
import glob
from sqlalchemy import text

# Caminho da pasta com os arquivos
pasta_input = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/07-Santander/Input/SantanderExtrato.xlsx'

# Ler o arquivo Excel ignorando as primeiras linhas de cabeçalho
df = pd.read_excel(pasta_input, dtype=str)

# Renomear colunas
df.columns = ["data_lanc", "descricao", "docto", "situacao", "credito", "debito", "saldo", "idcategoria", "observacao"]

#df["credito"] = df["credito"].str.replace(r"[^\d,.-]", "", regex=True).str.replace(".", "", regex=False)
df["credito"] = df["credito"].str.replace(r"[^\d,.-]", "", regex=True).str.replace(",", ".", regex=False)
df["debito"] = df["debito"].str.replace(r"[^\d,.-]", "", regex=True).str.replace(",", ".", regex=False)

# Converter para numérico
df["credito"] = pd.to_numeric(df["credito"], errors="coerce").fillna(0)
df["debito"] = pd.to_numeric(df["debito"], errors="coerce").fillna(0)

# Criar a coluna valor
df["valor"] = df["credito"] + df["debito"]

# converter a coluna 'data' para datetime
df['data_lanc'] = pd.to_datetime(df['data_lanc'], format='mixed', dayfirst=True)

# Remover colunas desnecessárias
df = df.drop(columns=["situacao", "saldo", "docto", "credito", "debito"])

#df["data_lanc"] = pd.to_datetime(df["data_lanc"], format="%d/%m/%Y", errors="coerce")

with engine.connect() as conn:
    # Verificar se a tabela existe no banco
    check_table_query = text("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables
            WHERE table_schema = 'fluxo'
            AND table_name = 'santander'
        );
    """)
    table_exists = conn.execute(check_table_query).scalar()

    if table_exists:
        # Se a tabela existir, deleta os dados
        delete_query = text(f"DELETE FROM fluxo.santander;")
        result = conn.execute(delete_query)
        df.to_sql('santander', engine, schema='fluxo', if_exists='append', index=False)
        conn.commit()  # Confirmar a exclusão
    else:
        # Inserir os novos dados
        df.to_sql('santander', engine, schema='fluxo', if_exists='append', index=False)

#df

In [ ]:
import pandas as pd
import os
import glob
from sqlalchemy import text

# Caminho da pasta com os arquivos
pasta_input = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/07-Santander/Input/SantanderExtrato.xlsx'

# Ler o arquivo Excel ignorando as primeiras linhas de cabeçalho
df = pd.read_excel(pasta_input, dtype=str)

# Renomear colunas
df.columns = ["data_lanc", "descricao", "docto", "situacao", "credito", "debito", "saldo", "idcategoria", "observacao"]

#df["credito"] = df["credito"].str.replace(r"[^\d,.-]", "", regex=True).str.replace(".", "", regex=False)
#df["credito"] = df["credito"].str.replace(r"[^\d,.-]", "", regex=True).str.replace(",", ".", regex=False)
#df["debito"] = df["debito"].str.replace(r"[^\d,.-]", "", regex=True).str.replace(",", ".", regex=False)

# Converter para numérico
#df["credito"] = pd.to_numeric(df["credito"], errors="coerce").fillna(0)
#df["debito"] = pd.to_numeric(df["debito"], errors="coerce").fillna(0)

# Criar a coluna valor
#df["valor"] = df["credito"] + df["debito"]

# converter a coluna 'data' para datetime
#df['data_lanc'] = pd.to_datetime(df['data_lanc'], format='mixed', dayfirst=True)

# Remover colunas desnecessárias
#df = df.drop(columns=["situacao", "saldo", "docto", "credito", "debito"])

#df["data_lanc"] = pd.to_datetime(df["data_lanc"], format="%d/%m/%Y", errors="coerce")


df

,data_lanc,descricao,docto,situacao,credito,debito,saldo,idcategoria,observacao
0,31/03/2025,REMUNERACAO APLICACAO AUTOMATICA,000000,NaN,0.02,NaN,203.72,Inv12,NaN
1,31/03/2025,PIX ENVIADO Bruno Abad ...,000000,NaN,NaN,-40,203.7,Laz06,churrasco
2,31/03/2025,PIX ENVIADO Rodrigo Morau,000000,NaN,NaN,-200,243.7,Outr06,transferencia entre contas
3,26/03/2025,REMUNERACAO APLICACAO AUTOMATICA,000000,NaN,0.01,NaN,443.7,Inv12,NaN
4,26/03/2025,PIX ENVIADO Rodrigo Morau,000000,NaN,NaN,-150,443.69,Outr06,transferencia entre contas
5,20/03/2025,REMUNERACAO APLICACAO AUTOMATICA,000000,NaN,0.01,NaN,593.69,Inv12,NaN
6,20/03/2025,PIX ENVIADO Rodrigo Morau,000000,NaN,NaN,-500,593.68,Outr06,transferencia entre contas
7,07/03/2025,REMUNERACAO APLICACAO AUTOMATICA,000000,NaN,0.02,NaN,1093.68,Inv12,NaN
8,07/03/2025,PAGAMENTO DE BOLETO OUTROS BANCOS HUBERT IMOV...,000000,NaN,NaN,-834.33,1093.66,Hab01,NaN
9,07/03/2025,PAGAMENTO DE BOLETO OUTROS BANCOS MORAES E OL...,000000,NaN,NaN,-397.42,1927.99,Edu05,NaN
